In [1]:
import time
from datetime import datetime

import pandas as pd
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [2]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
date_fr = "01/01/2022"
date_to = "02/03/2026"

In [4]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [5]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 2).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [6]:
records = pd.concat(dfs, axis=0, ignore_index=True)

In [7]:
records.empty

False

In [8]:
if records is not None and not records.empty:
    chunk_size = 1000
    for i in range(0, len(records), chunk_size):
        chunk = records.iloc[i : i + chunk_size]

        stmt = insert(reservation).values(chunk.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)
else:
    print("empty")

In [9]:
records.shape

(337792, 36)